In [1]:
import numpy as np
import pandas as pd
import gensim
import os

In [3]:
df = pd.read_csv('classification_data_gdelt.csv')

In [21]:
labels = df.Categorical_id

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [7]:
ar = []

In [ ]:
#with open("glove.840B.300d.txt", "rb") as lines:
#    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
#           for line in lines}

In [8]:
ar=[]
for i,row in df.iterrows():
    data = row['Content']
    data_tokens = word_tokenize(str(data).lower())
    data_tokens = [w for w in data_tokens if not w in stop_words]
    b = []
    for w in data_tokens:
        try:
            b.append(glove_model[w])
        except:
            b.append(np.zeros(300))
    #sentiment_score = np.array([row['sentiment_compound']])
    avg_vector = np.array([np.mean(b,axis=0)])
    ar.append(avg_vector)

In [ ]:
min_max_vector = np.concatenate((np.array([np.min(b,axis=0)]),np.array([np.max(b,axis=0)])),axis=1)

In [ ]:
ex = df.iloc[1]['Content']

In [ ]:
data_tokens = word_tokenize(str(ex).lower())
data_tokens = [w for w in data_tokens if not w in stop_words]

In [ ]:
a_v = np.array([np.mean([glove_model.wv[w] for w in data_tokens],axis=0)])[0]

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="glove.840B.300d.txt", word2vec_output_file="gensim_glove_vectors.txt")

In [2]:
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import cross_val_score


models = [
    LogisticRegression(C = 2.0,random_state=0),
]
CV = 10
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features_scaled, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [11]:
features = np.array(ar)

In [ ]:
from sklearn.preprocessing import normalize
normed_features = normalize(features, axis=1, norm='l1')

In [16]:
features = features.reshape((102442,300))

In [17]:
import sklearn.preprocessing as sk
from sklearn.preprocessing import scale
scaler = sk.MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(features)
features_scaled = scaler.transform(features)

In [27]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LogisticRegression    0.5
Name: accuracy, dtype: float64

In [ ]:
from sklearn import svm 

model = svm.SVC()

accuracies = cross_val_score(model, features_scaled, labels, scoring='accuracy', cv=CV)

for fold_idx, accuracy in enumerate(accuracies):
    entries.append(('SVM', fold_idx, accuracy))
cv_df_svm = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
cv_df_svm.groupby('model_name').accuracy.mean()

In [ ]:
w2v_model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [18]:
df['Categorical_id'] = ''

In [19]:
df.Categorical_id = pd.Categorical(df.Category)

In [20]:
df.Categorical_id = df.Categorical_id.cat.codes

In [ ]:
df.shape

In [ ]:
from sklearn.ensemble import BaggingClassifier
#bagging = BaggingClassifier(LogisticRegression(random_state=0),
#     max_samples=0.5, max_features=0.5)

In [ ]:
accuracies = cross_val_score(bagging, features_scaled, labels, scoring='accuracy', cv=CV)

In [ ]:
accuracies.mean()

In [ ]:
from sklearn import model_selection

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import cross_val_score
#model = ExtraTreesClassifier(n_estimators=100, max_features=7)
#kfold = model_selection.KFold(n_splits=10, random_state=2018)
#model = AdaBoostClassifier(n_estimators=100, random_state=2018)
model = GradientBoostingClassifier(n_estimators=100, random_state=2018)
bagging = BaggingClassifier(model,
                            max_samples=0.5, max_features=0.5)
accuracies = cross_val_score(bagging, features_scaled, labels, scoring='accuracy', cv=CV)

In [ ]:
accuracies.mean()

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn import svm 
lr = LogisticRegression(random_state=0)
ex_trees = ExtraTreesClassifier(n_estimators=100, max_features=7)
sv = svm.SVC(probability=True)
eclf = VotingClassifier(estimators=[('lr', lr), ('trees', ex_trees), ('sv', sv)], voting='soft',weights=[1,1,1])
accuracies = cross_val_score(eclf, features_scaled, labels, cv=10,)

In [ ]:
accuracies.mean()

In [9]:
from xgboost import XGBClassifier

In [23]:
from sklearn.model_selection import cross_val_score


model = XGBClassifier()
accuracies = cross_val_score(model, features_scaled, labels, cv=10)

In [25]:
accuracies

array([ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5])

In [ ]:
sv = svm.SVC(kernel='sigmoid')
accuracies = cross_val_score(sv, features_scaled, labels, cv=10)

In [ ]:
base_models = [ExtraTreesClassifier(n_estimators=100,max_features=7),
              XGBClassifier(),
              KNeighborsClassifier(n_neighbors=5)]

In [ ]:
blending_model = LogisticRegression(random_state=0)

In [ ]:
np.save('classification_features',features_scaled)

In [7]:
features_scaled = np.load('classification_features.npy')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
accuracies = cross_val_score(model, features_scaled, labels, cv=10)

In [ ]:
accuracies.mean()

In [ ]:
from mlxtend.classifier import StackingClassifier

In [ ]:
sc = StackingClassifier(classifiers=base_models, 
                          meta_classifier=blending_model)

In [ ]:
accuracies = cross_val_score(sc, features_scaled, labels, cv=10)

In [ ]:
accuracies.mean()

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=2.0, random_state=0)
lr.fit(features_scaled,labels)
fs = lr.predict_proba(features_scaled)[:,1]
class RandomForestClassifier_compability(lr):
    def predict(self, X):
        return self.predict_proba(X)[:, 1]
base_estimator = RandomForestClassifier_compability()
model = GradientBoostingClassifier(init=base_estimator,n_estimators=100, random_state=2018)
#neigh = KNeighborsClassifier(n_neighbors=5)
#lr = LogisticRegression(C=2.0, random_state=0)
#model = GradientBoostingClassifier(lr,n_estimators=100, random_state=2018)
accuracies = cross_val_score(model, features_scaled, labels, scoring='accuracy',cv=10)

TypeError: 'LogisticRegression' object is not callable

In [18]:
import xgboost as xgb

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.33, random_state=42)

In [19]:
xgb_clf = xgb.XGBClassifier()

In [36]:
xgb_clf.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [42]:
from xgboost import XGBClassifier, DMatrix
train_x_mat = DMatrix(X_train)

In [43]:
train_xgb_pred_mat = xgb_clf.get_booster().predict(train_x_mat,pred_leaf = True)

In [44]:
lr = LogisticRegression(C=2.0, random_state=0)

In [45]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
train_lr_feature_mat = one_hot_encoder.fit_transform(train_xgb_pred_mat)

In [46]:
lr.fit(train_lr_feature_mat,y_train)

LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [48]:
test_x_mat = DMatrix(X_test)

In [49]:
xgb_pred_mat = xgb_clf.get_booster().predict(test_x_mat,pred_leaf = True)

In [50]:
lr_feature = one_hot_encoder.transform(xgb_pred_mat)

In [51]:
y_pred = lr.predict(lr_feature)

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.78913793103448271

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
parameters = {'n_neighbors': [2,5,7,10]}
model = KNeighborsClassifier()
clf = GridSearchCV(model,parameters,cv=10)
clf.fit(features_scaled,labels)

In [ ]:
clf.best_params_

In [29]:
df

,Unnamed: 0,Category_id,Category,Content,Categorical_id
0,0,0,unrest,Bringing to an end the musical chairs game for...,1
1,1,0,unrest,MUMBAI: Request for a seat share landed a yout...,1
2,2,0,unrest,The CIA staff of the Mohali police on Saturday...,1
3,3,0,unrest,THE PREDICTION of snowfall on New Years Eve dr...,1
4,4,0,unrest,This was the year when old certainties crumble...,1
5,5,0,unrest,MUMBAI: The Enforcement Directorate (ED) is in...,1
6,6,0,unrest,VISAKHAPATNAM: Visakha Institute of Medical Sc...,1
7,7,0,unrest,The Samajwadi Party (SP) office and Chief Mini...,1
8,8,0,unrest,PATNA: The state police are making all possibl...,1
9,9,0,unrest,MADURAI: While many All India Anna Dravida Mun...,1
